In [1]:
# set up
import pandas as pd
import numpy as np
import pickle
import time 
import os
import matplotlib.pyplot as plt
import datetime
from operator import itemgetter
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pylab import *
from matplotlib.font_manager import FontProperties  
data_folder_path = 'C:\\Users\\goodluck\\Desktop\\DB'
raw_data_path =  "C:\\Users\\goodluck\\Desktop\\DB\\raw_data"
storage_path =  "C:\\Users\\goodluck\\Desktop\\DB\\database_storage"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
font = FontProperties(fname=r"simsun.ttf", size=14)  
matplotlib.rcParams['axes.unicode_minus'] =False
mpl.rcParams['font.sans-serif'] = ['SimHei']
tick_spacing = 4

# re-open
with open(DTBS_path, 'rb') as f:  
    DTBS = pickle.load(f)
    
def takeSecond(elem):
    return elem[1]


def retrieve_low_nodes(codes,dt,num):
    dl_rank = []
    for code in codes:
        if DTBS['A'][code][dt]['dl'] != 999:
            dl_rank.append((code, DTBS['A'][code][dt]['dl']))
    
    dl_rank.sort(key=takeSecond)
    low = dl_rank[0:num]
    return [i[0] for i in low]

def past_qs(days, code):
    
    for day in days:
            
        if DTBS['A'][code][day]['qs'] == 1:
            return 1
    
    return 0

In [2]:
dt = DTBS['D']['day'][-1]
print(dt)
dt = DTBS['D']['day'][-1]
past_dt = DTBS['D']['day'][-20:]
re = retrieve_low_nodes(DTBS['A'].keys(), dt , 400)


2023-04-17


In [7]:
for r in re:
    print(r, DTBS['B'][r]['cn'], '过去十天满足过强赎条件',past_qs(past_dt, r), '收盘价', DTBS['A'][r][dt]['dp'], '转股溢价率',DTBS['A'][r][dt]['cpr'], '双低',DTBS['A'][r][dt]['dl'], DTBS['B'][r]['cat1'], DTBS['B'][r]['cat2'])

128114.SZ 正邦转债 过去十天满足过强赎条件 0 收盘价 91.827 转股溢价率 2.28 双低 94.107 农林牧渔(2021) 养殖业(2021)
128100.SZ 搜特转债 过去十天满足过强赎条件 0 收盘价 78.14 转股溢价率 22.57 双低 100.71000000000001 纺织服饰(2021) 服装家纺(2021)
111006.SH 嵘泰转债 过去十天满足过强赎条件 1 收盘价 111.269 转股溢价率 1.23 双低 112.49900000000001 汽车(2021) 汽车零部件(2021)
113044.SH 大秦转债 过去十天满足过强赎条件 0 收盘价 113.995 转股溢价率 2.52 双低 116.515 交通运输(2021) 铁路公路(2021)
110053.SH 苏银转债 过去十天满足过强赎条件 0 收盘价 123.206 转股溢价率 3.6 双低 126.806 银行(2021) 城商行Ⅱ(2021)
110043.SH 无锡转债 过去十天满足过强赎条件 0 收盘价 112.059 转股溢价率 15.13 双低 127.189 银行(2021) 农商行Ⅱ(2021)
113595.SH 花王转债 过去十天满足过强赎条件 0 收盘价 128.736 转股溢价率 -0.9 双低 127.83599999999998 建筑装饰(2021) 基础建设(2021)
123077.SZ 汉得转债 过去十天满足过强赎条件 1 收盘价 128.8 转股溢价率 -0.25 双低 128.55 计算机(2021) 白酒Ⅱ(2021)
128034.SZ 江银转债 过去十天满足过强赎条件 0 收盘价 112.36 转股溢价率 16.88 双低 129.24 银行(2021) 农商行Ⅱ(2021)
113013.SH 国君转债 过去十天满足过强赎条件 0 收盘价 104.961 转股溢价率 24.43 双低 129.391 非银金融(2021) 证券Ⅱ(2021)
128081.SZ 海亮转债 过去十天满足过强赎条件 0 收盘价 126.866 转股溢价率 5.94 双低 132.806 有色金属(2021) 工业金属(2021)
128048.SZ 张行转债 过去十天满足过强赎条件 0 收盘价 115.719 转股溢价率 1

In [ ]:
+ 嵘泰转债
鹰19转债
贵燃转债
现代转债
现代转债

-皖天
南银
孚日
苏行
文科

In [ ]:
for day in DTBS['D']['day'][1000:]:
    print(day, DTBS['A']['123077.SZ'][day]['qs'])
    

In [ ]:
days = days[:len(days)-1]
days[1000:]

In [ ]:
days.append('2023-04-04')

In [ ]:
days.append('2023-04-06')

In [36]:
DTBS['D']['day'] = days

NameError: name 'days' is not defined

In [28]:
days[1000:]

['2021-02-09',
 '2021-02-10',
 '2021-02-18',
 '2021-02-19',
 '2021-02-22',
 '2021-02-23',
 '2021-02-24',
 '2021-02-25',
 '2021-02-26',
 '2021-03-01',
 '2021-03-02',
 '2021-03-03',
 '2021-03-04',
 '2021-03-05',
 '2021-03-08',
 '2021-03-09',
 '2021-03-10',
 '2021-03-11',
 '2021-03-12',
 '2021-03-15',
 '2021-03-16',
 '2021-03-17',
 '2021-03-18',
 '2021-03-19',
 '2021-03-22',
 '2021-03-23',
 '2021-03-24',
 '2021-03-25',
 '2021-03-26',
 '2021-03-29',
 '2021-03-30',
 '2021-03-31',
 '2021-04-01',
 '2021-04-02',
 '2021-04-06',
 '2021-04-07',
 '2021-04-08',
 '2021-04-09',
 '2021-04-12',
 '2021-04-13',
 '2021-04-14',
 '2021-04-15',
 '2021-04-16',
 '2021-04-19',
 '2021-04-20',
 '2021-04-21',
 '2021-04-22',
 '2021-04-23',
 '2021-04-26',
 '2021-04-27',
 '2021-04-28',
 '2021-04-29',
 '2021-04-30',
 '2021-05-06',
 '2021-05-07',
 '2021-05-10',
 '2021-05-11',
 '2021-05-12',
 '2021-05-13',
 '2021-05-14',
 '2021-05-17',
 '2021-05-18',
 '2021-05-19',
 '2021-05-20',
 '2021-05-21',
 '2021-05-24',
 '2021-05-

In [30]:
DTBS['D']['mon'] = DTBS['D']['mon'][:len(DTBS['D']['mon'])-1]

In [33]:
DTBS['D']['tue'] = DTBS['D']['tue'][:len(DTBS['D']['tue'])-1]

In [34]:
DTBS['D']['tue']

['2017-01-03',
 '2017-01-10',
 '2017-01-17',
 '2017-01-24',
 '2017-02-07',
 '2017-02-14',
 '2017-02-21',
 '2017-02-28',
 '2017-03-07',
 '2017-03-14',
 '2017-03-21',
 '2017-03-28',
 '2017-04-11',
 '2017-04-18',
 '2017-04-25',
 '2017-05-02',
 '2017-05-09',
 '2017-05-16',
 '2017-05-23',
 '2017-06-06',
 '2017-06-13',
 '2017-06-20',
 '2017-06-27',
 '2017-07-04',
 '2017-07-11',
 '2017-07-18',
 '2017-07-25',
 '2017-08-01',
 '2017-08-08',
 '2017-08-15',
 '2017-08-22',
 '2017-08-29',
 '2017-09-05',
 '2017-09-12',
 '2017-09-19',
 '2017-09-26',
 '2017-10-10',
 '2017-10-17',
 '2017-10-24',
 '2017-10-31',
 '2017-11-07',
 '2017-11-14',
 '2017-11-21',
 '2017-11-28',
 '2017-12-05',
 '2017-12-12',
 '2017-12-19',
 '2017-12-26',
 '2018-01-02',
 '2018-01-09',
 '2018-01-16',
 '2018-01-23',
 '2018-01-30',
 '2018-02-06',
 '2018-02-13',
 '2018-02-27',
 '2018-03-06',
 '2018-03-13',
 '2018-03-20',
 '2018-03-27',
 '2018-04-03',
 '2018-04-10',
 '2018-04-17',
 '2018-04-24',
 '2018-05-08',
 '2018-05-15',
 '2018-05-

In [35]:
# save
f_save = open(DTBS_path, 'wb')
pickle.dump(DTBS, f_save)
f_save.close()


In [35]:
def select_condition(codes, dt, fcd):
    satisfy = []
    if fcd[0] == 'lp':
        lm = calculate_time(dt, 'p', 20)
        
        for code in codes:
            if low_prc(code, dt, lm, fcd[1]):
                satisfy.append(code)
        
    else:
        for code in codes:
            if DTBS['A'][code][dt][fcd[0]] <= fcd[2] and DTBS['A'][code][dt][fcd[0]] >= fcd[1]:
                satisfy.append(code)
                    
    return satisfy

dt = DTBS['D']['day'][-1]
print(dt)
select = list(DTBS['A'].keys())

flt_cd = [['lp', 0.5], ['dp', 110, 130], ['bl',0, 3], ['cpr',-999, 50]]
for fcd in flt_cd:
    select = select_condition(select, dt, fcd)
    
    

2023-04-17


In [36]:
len(select)

13

In [37]:
def rank(codes, dt, idx, num, d):
    rk = []
    if num == -1:
        num = len(codes)
    for code in codes:
        if idx == 'lp':
            if not np.isnan(DTBS['A'][code][dt]['dp']):
                lm = calculate_time(dt, 'p', 20)
                prc = get_prcs(code, lm)
                lp = absolute_low(prc, 0)
                rk.append((code, DTBS['A'][code][dt]['dp']/lp))
            
            
        elif idx != 'dl':
            if not np.isnan(DTBS['A'][code][dt][idx]):
                rk.append((code, DTBS['A'][code][dt][idx]))
        else:
            if not np.isnan(DTBS['A'][code][dt][idx]):
                if DTBS['A'][code][dt][idx] != 999:
                    rk.append((code, DTBS['A'][code][dt][idx]))
    if num >= len(rk):
        num = len(rk)
        
     
#     reverse = True 降序
    if d == 'd':
        rk.sort(key=takeSecond, reverse = True)
    else:
        rk.sort(key=takeSecond, reverse = False)

    if idx == 'lp':
        lp = [i for i in rk if i[1] == 1.0]
        ot = []
        if num - len(lp) > 0:
            ot = rk[len(lp):num]
        
        low = lp + ot
        
    else:    
        low = rk[0:num]
#     print("In",dt , len(low), "of", len(codes),  "is selected")   

#     print(low)
    
    
    return [i[0] for i in low]
  

def get_prcs(code, dts):
    prc = []
    for dt in dts:
        prc.append(DTBS['A'][code][dt]['dp'])
    prc.sort()
    return prc
    
def relative_low(lst, pos):
    return np.quantile(a=lst, q = pos)
    
def absolute_low(lst, pos):
    return min(lst) + pos * (max(lst) - min(lst))

def low_prc(code, dt, lm, pos):
    prc = get_prcs(code, lm)
#     lp = min([relative_low(prc, pos), absolute_low(prc, pos)])
    lp = absolute_low(prc, pos)
    tp = DTBS['A'][code][dt]['dp']
#     if (tp <= lp):
#         print(code, tp, lp)
#     if tp <= lp:
#         print(tp, lp)
    return tp <= lp

def calculate_time(dt, d, l):
    time = []
    days = DTBS['D']['day']
    if d == 'p':
        for i in range(len(days)):
            if days[i] == dt:
                break
                
    return days[i-l+1:i+1]
            
rk_cd = [['lp', 28, 'a']]
lp = rank(select, dt, rk_cd[0][0], rk_cd[0][1], rk_cd[0][2])
len(lp)


13

In [34]:
print("当前日期：", dt)
for r in lp:
    print(r, DTBS['B'][r]['cn'], '满足强赎天数：' , past_qs(past_dt, r), "收盘价：", DTBS['A'][r][dt]['dp'],  "转股溢价率：", DTBS['A'][r][dt]['cpr'], "到期收益率：", DTBS['A'][r][dt]['ytm'],"换手率：", DTBS['A'][r][dt]['trt'] )

当前日期： 2023-04-17
123166.SZ 蒙泰转债 满足强赎天数： 0 收盘价： 129.935 转股溢价率： 31.39 到期收益率： -1.23 换手率： 1.13
123011.SZ 德尔转债 满足强赎天数： 0 收盘价： 118.8 转股溢价率： 117.97 到期收益率： -3.41 换手率： 1.18
123039.SZ 开润转债 满足强赎天数： 0 收盘价： 116.454 转股溢价率： 127.42 到期收益率： 0.85 换手率： 1.43
128021.SZ 兄弟转债 满足强赎天数： 0 收盘价： 121.025 转股溢价率： 22.44 到期收益率： -20.15 换手率： 8.75
113573.SH 纵横转债 满足强赎天数： 0 收盘价： 118.333 转股溢价率： 70.02 到期收益率： 1.12 换手率： 1.78
128033.SZ 迪龙转债 满足强赎天数： 0 收盘价： 124.332 转股溢价率： 25.24 到期收益率： -21.18 换手率： 6.91
113030.SH 东风转债 满足强赎天数： 0 收盘价： 121.826 转股溢价率： 38.27 到期收益率： -2.06 换手率： 1.78
113565.SH 宏辉转债 满足强赎天数： 0 收盘价： 120.153 转股溢价率： 42.32 到期收益率： -1.24 换手率： 2.08
128039.SZ 三力转债 满足强赎天数： 0 收盘价： 119.892 转股溢价率： 50.71 到期收益率： -9.21 换手率： 1.6
113574.SH 华体转债 满足强赎天数： 0 收盘价： 116.015 转股溢价率： 147.07 到期收益率： -0.6 换手率： 1.24
113546.SH 迪贝转债 满足强赎天数： 0 收盘价： 122.472 转股溢价率： 37.18 到期收益率： -1.29 换手率： 2.71
123159.SZ 崧盛转债 满足强赎天数： 0 收盘价： 124.154 转股溢价率： 40.99 到期收益率： -0.74 换手率： 0.87
113566.SH 翔港转债 满足强赎天数： 0 收盘价： 123.006 转股溢价率： 47.85 到期收益率： -0.29 换手率： 1.87
123065.SZ 宝莱转债 满足强赎天数：

In [5]:
dt = DTBS['D']['day'][-1]
dt

'2023-04-11'

In [6]:
lm =  DTBS['D']['day'][-20:]

In [7]:
len(lm)

20

In [61]:

# code = '118016.SH'
# prc = []
# for dt in lm:
#     prc.append(DTBS['A'][code][dt]['dp'])
# prc.sort()    
# max_prc = max(prc)
# min_prc = min(prc)
# print(prc)
# print(max_prc, min_prc)

def get_prcs(code, dts):
    prc = []
    for dt in dts:
        prc.append(DTBS['A'][code][dt]['dp'])
    prc.sort()
    return prc
    
def relative_low(lst, pos):
    return np.quantile(a=lst, q = pos)
    
def absolute_low(lst, pos):
    return min(lst) + pos * (max(lst) - min(lst))

def low_prc(code, dt, lm, pos):
    prc = get_prcs(code, lm)
#     lp = min([relative_low(prc, pos), absolute_low(prc, pos)])
    lp = absolute_low(prc, pos)
    tp = DTBS['A'][code][dt]['dp']
#     print(code, tp, lp, tp <= lp)
#     if tp <= lp:
#         print(tp, lp)
    return tp <= lp

In [62]:
113600.SH

SyntaxError: invalid decimal literal (3743447309.py, line 1)

In [63]:
max([1,2,3])

3

In [64]:
low_prc('123128.SZ', DTBS['D']['day'][-1], DTBS['D']['day'][-20:], 0.2)


123128.SZ 102.006 100.96039999999999 False


False

In [65]:
print(min(prc))
print(np.quantile(a=prc, q=1))

NameError: name 'prc' is not defined

In [66]:


relative_low(prc, 0.20)
absolute_low(prc, 0.20)

NameError: name 'prc' is not defined

In [67]:
lm =  DTBS['D']['day'][-22:]
dt = DTBS['D']['day'][-1]
pos = 0
low_code = []
for code in DTBS['A'].keys():
    flag = 0
    for odt in lm:
        if DTBS['A'][code][odt]['ia'] == 0:
            flag = 1
    if flag == 0:
        if low_prc(code, dt, lm, pos):
            low_code.append(code)
    

print(len(low_code))

lowlow = []
for code in low_code:
    if DTBS['A'][code][dt]['bl'] <3:
        lowlow.append(code)
        
print(len(lowlow))
print(lowlow)

110088.SH 117.81 117.167 False
113545.SH 123.773 122.283 False
113631.SH 118.935 117.871 False
123128.SZ 102.006 100.699 False
123156.SZ 139.466 133.329 False
127027.SZ 120.231 119.58 False
127061.SZ 111.198 108.911 False
110047.SH 118.643 118.643 True
110063.SH 118.679 118.679 True
110070.SH 118.164 118.164 True
110087.SH 113.624 111.271 False
110089.SH 114.688 114.42 False
110092.SH 119.269 118.608 False
111001.SH 115.08 114.5 False
111002.SH 129.639 127.84700000000001 False
111003.SH 125.743 125.743 True
111007.SH 173.409 163.05700000000002 False
111012.SH 127.939 126.169 False
113025.SH 239.829 237.247 False
113027.SH 168.16 164.486 False
113046.SH 107.369 105.929 False
113047.SH 125.812 122.935 False
113058.SH 116.399 116.399 True
113059.SH 117.488 114.686 False
113064.SH 135.554 135.554 True
113534.SH 270.301 266.394 False
113535.SH 113.647 112.698 False
113566.SH 122.751 122.751 True
113591.SH 127.496 123.654 False
113600.SH 120.273 118.937 False
113615.SH 288.669 261.246 False


In [49]:
print(low_code)

['110088.SH', '113631.SH', '123128.SZ', '113046.SH', '113535.SH', '113623.SH', '113641.SH', '118000.SH', '118005.SH', '118010.SH']


In [30]:
DTBS['A'][code][dt]['dp']

117.89200000000001

In [40]:
DTBS['A']['113600.SH'][DTBS['D']['day'][-1]]['dp']

119.941

In [88]:



low_prc('128105.SZ', DTBS['D']['day'][-1], DTBS['D']['day'][-20:], 0.2)



128105.SZ 106.208 106.4964 106.6056 106.4964


True

In [22]:
dt = DTBS['D']['day'][-1]
can = []
for code in DTBS['A'].keys():

    if DTBS['A'][code][dt]['ia'] == 1:
#         print(DTBS['A'][code][dt]['bl'])
        if DTBS['A'][code][dt]['bl'] <= 7:
            if DTBS['A'][code][dt]['trt'] >= 2:
                if DTBS['A'][code][dt]['trt'] <= 10:
                    if DTBS['A'][code][dt]['ytm'] >= -1:
                        can.append(code)

print(len(can))
re = retrieve_low_nodes(can, dt , 400)

26


In [27]:
# re = retrieve_low_nodes(lb, dt , 50)
past_dt = DTBS['D']['day'][-10:]
for r in re:
    print(r,  DTBS['B'][r]['cn'],  '过去满足强赎',past_qs(past_dt, r),  '收盘价',  DTBS['A'][r][dt]['dp'], '双低', DTBS['A'][r][dt]['dl'], 'ytm', DTBS['A'][r][dt]['ytm'] ,'换手率',DTBS['A'][r][dt]['trt'], '市值', DTBS['A'][r][dt]['bl'])

123141.SZ 宏丰转债 过去满足强赎 0 收盘价 123.8 双低 146.24 ytm -0.44 换手率 4.28 市值 3.211
127054.SZ 双箭转债 过去满足强赎 0 收盘价 118.35 双低 151.75 ytm -0.28 换手率 2.63 市值 5.136
113628.SH 晨丰转债 过去满足强赎 0 收盘价 118.594 双低 153.444 ytm 0.51 换手率 2.43 市值 4.15
123175.SZ 百畅转债 过去满足强赎 0 收盘价 121.55 双低 154.35 ytm -0.5 换手率 3.84 市值 4.2
111012.SH 福新转债 过去满足强赎 0 收盘价 127.939 双低 155.059 ytm -0.92 换手率 2.05 市值 4.29
127059.SZ 永东转2 过去满足强赎 0 收盘价 118.509 双低 155.369 ytm 0.27 换手率 3.46 市值 3.792
113665.SH 汇通转债 过去满足强赎 0 收盘价 117.934 双低 155.994 ytm 0.36 换手率 2.55 市值 3.6
110070.SH 凌钢转债 过去满足强赎 0 收盘价 118.164 双低 158.554 ytm -0.74 换手率 4.22 市值 2.171
123100.SZ 朗科转债 过去满足强赎 0 收盘价 119.09 双低 159.74 ytm -0.58 换手率 2.09 市值 3.797
123153.SZ 英力转债 过去满足强赎 0 收盘价 117.891 双低 161.721 ytm 0.59 换手率 4.32 市值 3.4
113604.SH 多伦转债 过去满足强赎 0 收盘价 116.001 双低 162.591 ytm 0.08 换手率 4.18 市值 6.396
128143.SZ 锋龙转债 过去满足强赎 0 收盘价 123.0 双低 163.97 ytm -0.56 换手率 2.92 市值 2.44
118011.SH 银微转债 过去满足强赎 0 收盘价 125.871 双低 164.981 ytm -0.7 换手率 2.21 市值 4.997
123154.SZ 火星转债 过去满足强赎 0 收盘价 125.268 双低 168.5679999999